# Quantum Fourier Transform and Quantum Phase Esitmation with Tequila

In this tutorial, we introduce the quantum fourier transform (QFT) and the quamtum phase estimation (QPE) algorithm, derive the circuit, and implement it using Tequila.

## Contents
1. [Introduction](#introduction)
2. [Quantum Fourier Transform](#qft)     
    2.1 [Mathematical Description of the Quantum Fourier Transform](#math)         
    2.2 [Expansion of the Quantum Fourier Transform](#expansion)  
    2.3 [The Derivation of the Circuit Implementation](#deriv)  
    2.4 [Tequila Implementation of the Quantum Fourier Transform](#impl)  
    2.5 [Simulating the QFT Circuit](#sim)
3. [Quantum Phase Estimation](#qpe)     
    3.1 [Mathematical Description of the Quantum Phase Estimation](#math2)  
    3.2 [Quantum Phase Estimation Circuit](#sim2)
4. [References](#references)

# 1. Introduction <a id='introduction'></a>

The Fourier transform occurs in many different places throughout classical computing, ranging from signal processing applications to data compression to complexity theory. The quantum Fourier transform (QFT) is the quantum counterpart of the discrete Fourier transform over the amplitudes of a wavefunction. It is part of many quantum algorithms, such as Shor's factoring algorithm and quantum phase estimation. As an example of where quantum Fourier transform is used, we will also take a look at the quantum phase estimation algorithm, and how it utilizes the QFT. Quantum phase estimation is also one of the most important subroutines in quantum computation. It serves as a central building block for many quantum algorithms, such as Shor's algorithm and Quantum Counting.

# 2. Quantum Fourier Transform <a id='qft'></a>

## 2.1 Mathematical Description of the Quantum Fourier Transform <a id='math'></a>
The Quantum Fourier Transform acts on a quantum state 
$$ 
\lvert x \rangle = \sum_{i = 0}^{N-1} x_i \lvert i \rangle 
$$
to give the quantum state 
$$
\lvert y \rangle = \sum_{i = 0}^{N-1} y_i \lvert i \rangle
$$
such that 
$$
y_i = \frac{1}{\sqrt{N}} \sum_{j = 0}^{N-1}x_je^{2\pi i \frac{ji}{N}} = \frac{1}{\sqrt{N}} \sum_{j = 0}^{N-1}x_jw^{ji}_{N}
$$
We note that this mapping is equivalent to applying to each basis state $\lvert i \rangle$ the following mapping:
$$
\lvert i \rangle \mapsto \frac{1}{\sqrt{N}}\sum_{y = 0}^{N-1}w^{iy}_N\lvert y \rangle
$$
To see this equivalance, expand and combine liked terms, then it should be clear. Moreover, this mapping correponds to the following Unitary Matrix $U_{QFT}$, where:
$$
U_{QFT} = \frac{1}{\sqrt{N}}\sum_{i=0}^{N-1}\sum_{y=0}^{N-1} w^{iy}_N \lvert y \rangle \langle i \rvert 
$$
To see this that this matrix represents the transformation as defined above and is Unitary, we note that:
\begin{align*}
U_{QFT}\lvert j \rangle &= \Big(\frac{1}{\sqrt{N}}\sum_{i=0}^{N-1}\sum_{y=0}^{N-1} w^{iy}_N \lvert y \rangle \langle i \rvert\Big) \lvert j \rangle\\
&= \frac{1}{\sqrt{N}}\sum_{y=0}^{N-1} \lvert y \rangle \sum_{i=0}^{N-1} w^{iy}_N  \langle i \mid j \rangle\\
&= \frac{1}{\sqrt{N}}\sum_{y=0}^{N-1} \lvert y \rangle \sum_{i=0}^{N-1} w^{iy}_N  \delta_{ij} && \text{Since $(\lvert 0 \rangle, \dots, \lvert {N-1} \rangle)$ is an orthonormal basis}\\
&= \frac{1}{\sqrt{N}}\sum_{y=0}^{N-1} w^{jy}_N \lvert y \rangle \\
\end{align*}

as wanted. To see that $U_{QFT}$ is Unitary, we note that:
$$
U_{QFT}^{\dagger} = \frac{1}{\sqrt{N}}\sum_{i'=0}^{N-1}\sum_{y'=0}^{N-1} w^{(-1)i'y'}_N \lvert i' \rangle \langle y' \rvert 
$$
then we get that:
\begin{align*}
U_{QFT}^{\dagger}U_{QFT} &= \frac{1}{N} \sum_{i, y, i', y'} e^{2\pi i (i'y' - iy)/N}\rvert y \rangle \langle y' \rvert \delta_{ii'}\\
&= \frac{1}{N} \sum_{y, i, y'} e^{2\pi i (y' - y)i/N}\rvert y \rangle \langle y' \lvert \\
&= \frac{1}{N} \sum_{y, i, y'} \rvert y \rangle \langle y' \lvert \delta_{yy'}\\
&= \sum_y\lvert y \rangle \langle y \rvert = I
\end{align*}
as desired. Hence, $U_{QFT}$ is unitary. 

Now we attempt to develop a Quantum Circuit in Python that will perform the $U_{QFT}$ transformation.

## 2.2 Expansion of the Quantum Fourier Transform$^1$ <a id='expansion'></a>
To see why this circuit implements the Quantum Fourier Transform, we first consider the expansion of $U_{QFT}$ as it operates on the state $\lvert x \rangle = \lvert x_1\dots x_n \rangle$, where each $x_i$ represents a qubit, thus $N = 2^n$:
$$
\begin{aligned}
QFT_N\vert x \rangle & = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1}\omega_N^{xy} \vert y \rangle 
\\
& = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} e^{2 \pi i xy / 2^n} \vert y \rangle ~\text{since}\: \omega_N^{xy} = e^{2\pi i \frac{xy}{N}} \:\text{and}\: N = 2^n 
\\
& = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} e^{2 \pi i \left(\sum_{k=1}^n y_k/2^k\right) x} \vert y_1 \ldots y_n \rangle \:\text{rewriting in fractional binary notation}\: y = y_1\ldots y_n, y/2^n = \sum_{k=1}^n y_k/2^k 
\\
& = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} \prod_{k=1}^n e^{2 \pi i x y_k/2^k } \vert y_1 \ldots y_n \rangle \:\text{after expanding the exponential of a sum to a product of exponentials} 
\\
& = \frac{1}{\sqrt{N}} \bigotimes_{k=1}^n  \left(\vert0\rangle + e^{2 \pi i x /2^k } \vert1\rangle \right) \:\text{after rearranging the sum and products, and expanding} 
\sum_{y=0}^{N-1} = \sum_{y_1=0}^{1}\sum_{y_2=0}^{1}\ldots\sum_{y_n=0}^{1} 
\\
& = \frac{1}{\sqrt{N}}
\left(\vert0\rangle + e^{\frac{2\pi i}{2}x} \vert1\rangle\right) 
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^2}x} \vert1\rangle\right) 
\otimes  
\ldots
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^{n-1}}x} \vert1\rangle\right) 
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^n}x} \vert1\rangle\right) 
\end{aligned}
$$

## 2.3 The Derivation of the Circuit Implementation <a id='deriv'></a>

### 2.3.1 Hadamard and Controlled Rotational Gates
In this section, we will look at how the circuit implemented above equates the the $QFT_N$ matrix mathematically. Looking at the code, we see that there are only two gates that are used: The single-qubit Hadamard gate, $H$, and the two-qubit controlled rotation $CROT_k$ gate. The Hadamard gate is the following:
$$
H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
$$
and controlled rotation $CROT_k$ given in block-diagonal form as: 

$$CROT_k =
\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & \exp\left(\frac{2\pi i}{2^k}\right)\end{bmatrix}
$$
Now we note that applying the $H$ operator to a single-qubit state $\vert\psi\rangle = \alpha \vert 0 \rangle + \beta \vert 1 \rangle$, we get a new state:
$$\frac{1}{\sqrt{2}}(\alpha + \beta) \vert 0 \rangle + \frac{1}{\sqrt{2}}(\alpha - \beta)  \vert 1 \rangle $$
letting $\vert\psi\rangle = \vert0\rangle$, we get $\alpha = 1, \beta = 0$, therefore:
$$
H\vert\psi\rangle = H\vert0\rangle = \frac{1}{\sqrt{2}}(\vert0\rangle + \vert1\rangle)
$$
and similarly when  $\vert\psi\rangle = \vert1\rangle$, we get:
$$
H\vert\psi\rangle = H\vert1\rangle = \frac{1}{\sqrt{2}}(\vert0\rangle - \vert1\rangle)
$$
Hence, in general, we can say:
$$
H\vert \psi \rangle = \frac{1}{\sqrt{2}}\left(\vert0\rangle + \exp\left(\frac{2\pi i}{2}\psi\right)\vert1\rangle\right)
$$
Now we look at how the $CROT_k$ gate acts on the $2-$qubit system $\lvert\psi\rangle = \alpha\lvert00\rangle + \beta\lvert01\rangle + \gamma\lvert10\rangle + \delta\lvert11\rangle:$
$$
CROT_k\lvert\psi\rangle = \lvert\psi\rangle = \alpha\lvert00\rangle + \beta\lvert01\rangle + \gamma\lvert10\rangle + \exp\left(\frac{2\pi i}{2^k}\right)\delta\lvert11\rangle
$$
Then it is easy to see that:
$$CROT_k\vert 0\psi_j\rangle = \vert 0\psi_j\rangle$$


and


$$CROT_k\vert 1\psi_j\rangle = \exp\left( \frac{2\pi i}{2^k}\psi_j \right)\vert 1\psi_j\rangle$$

Let us apply these two gates to get an equivalent tranform as the Unitary Matrix $U_{QFT}$ as defined above.



### 2.3.2 Equivalance of Circuit and $U_{QFT}$ Transformation Proof$^1$
The circuit operates as follows. We start with an n-qubit input state $\vert x_1x_2\ldots x_n\rangle$.


After the first Hadamard gate on qubit 1, the state is transformed from the input state to 

$$
H_1\vert x_1x_2\ldots x_n\rangle = 
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + \exp\left(\frac{2\pi i}{2}x_1\right)\vert1\rangle\right]
\otimes
\vert x_2x_3\ldots x_n\rangle
$$

<li> After the $UROT_2$ gate on qubit 1 controlled by qubit 2, the state is transformed to

$$
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + \exp\left(\frac{2\pi i}{2^2}x_2 + \frac{2\pi i}{2}x_1\right)\vert1\rangle\right]
\otimes
\vert x_2x_3\ldots x_n\rangle
$$

<li> After the application of the last $UROT_n$ gate on qubit 1 controlled by qubit $n$, the state becomes

$$
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^n}x_n + 
\frac{2\pi i}{2^{n-1}}x_{n-1} + 
\ldots + 
\frac{2\pi i}{2^2}x_2 + 
\frac{2\pi i}{2}x_1
\right)
\vert1\rangle\right]
\otimes
\vert x_2x_3\ldots x_n\rangle
$$

Letting

$$
x = 2^{n-1}x_1 + 2^{n-2}x_2 + \ldots + 2^1x_{n-1} + 2^0x_n
$$

we can write the above state as 

$$
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^n}x 
\right)
\vert1\rangle\right]
\otimes
\vert x_2x_3\ldots x_n\rangle
$$

<li> After the application of a similar sequence of gates for qubits $2\ldots n$, we find the final state to be:

$$
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^n}x 
\right)
\vert1\rangle\right]
\otimes
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^{n-1}}x 
\right)
\vert1\rangle\right]
\otimes
\ldots
\otimes
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^{2}}x 
\right)
\vert1\rangle\right]
\otimes
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^{1}}x 
\right)
\vert1\rangle\right]
$$

which is exactly the QFT of the input state as derived above but in reversed order, which is why the swaps in the circuit is applied.
    
## 2.4 Tequila Implementation of the Quantum Fourier Transform <a id='impl'></a>

In [18]:
import tequila as tq
from numpy import pi

def qft_rotations(n):
    """Given the number of qubit registers <n>, create the appropriate Controlled Rotation gates"""
    circuit = tq.gates.H(target=0)
    for i in range(1, n + 1):
        for qubit in reversed(range(i)):
            circuit = tq.gates.Phase(phi = pi/(2 ** (i - qubit)), target = i, control = qubit) + circuit
        circuit = tq.gates.H(target=i) + circuit
    return circuit

def qft_swap(circuit, n):
    """Given the number of qubit registers <n> and the circuit <circuit>, create the appropriate Controlled Rotation gates,
    perform appropriate register swaps"""
    for qubit in range(n//2):
        circuit += tq.gates.SWAP(qubit, n-qubit - 1)
    return circuit

def qft(n):
    """Given the number of qubit registers <n>, return the QFT circuit."""
    return qft_swap(qft_rotations(n - 1), n)

circuit = qft(3)
tq.draw(circuit, backend='qiskit')

     ┌─────────┐                                 ┌─────────┐                 »
q_0: ┤ RZ(π/8) ├─────────────■────────────────■──┤ RZ(π/4) ├─────────────────»
     ├─────────┤             │                │  └─────────┘                 »
q_1: ┤ RZ(π/4) ├─────────────┼────────────────┼───────────────■──────────────»
     └──┬───┬──┘┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐»
q_2: ───┤ H ├───┤ RZ(π/8) ├┤ X ├┤ RZ(-π/8) ├┤ X ├┤ RZ(π/4) ├┤ X ├┤ RZ(-π/4) ├»
        └───┘   └─────────┘└───┘└──────────┘└───┘└─────────┘└───┘└──────────┘»
c: 3/════════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                ┌───┐   
«q_0: ───────────────────────■────────────────■──┤ H ├─X─
«          ┌───┐┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐└───┘ │ 
«q_1: ──■──┤ H ├┤ RZ(π/4) ├┤ X ├┤ RZ(-π/4) ├┤ X ├──────┼─
«     ┌─┴─┐└───┘└─────────┘└───┘└──────────┘└───┘      │ 

## 2.5 Simulating the QFT Circuit <a id='sim'></a>

In order to check the correctness of our circuit, we will take the Quantum Fourier state of the bits representing the number $4$ and then take the inverse of our circuit and simulate the circuit on real hardware. If we see that the highest probability is placed on $3$ (ie. $011$), then we can be sure our circuit is correct. Let us first define our qft_inverse function:

In [19]:
tq.draw(qft(3).dagger())
    

           ┌───┐    ┌──────────┐                     ┌──────────┐            »
q_0: ─X────┤ H ├────┤ RZ(-π/4) ├──■───────────────■──┤ RZ(-π/8) ├────────────»
      │ ┌──┴───┴───┐└──────────┘┌─┴─┐┌─────────┐┌─┴─┐└──┬───┬───┘┌──────────┐»
q_1: ─┼─┤ RZ(-π/4) ├────────────┤ X ├┤ RZ(π/4) ├┤ X ├───┤ H ├────┤ RZ(-π/4) ├»
      │ ├──────────┤            └───┘└─────────┘└───┘   └───┘    └──────────┘»
q_2: ─X─┤ RZ(-π/4) ├─────────────────────────────────────────────────────────»
        └──────────┘                                                         »
c: 3/════════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                                
«q_0: ───────────────────────────────────■───────────────■───────
«                                        │               │       
«q_1: ──■───────────────■────────────────┼───────────────┼───────
«     ┌─┴─┐┌─────────┐┌─┴

Now we get the Quantum Fourier state for $011$, which rotates the most significant qubit by $\frac{3}{4\pi}$, the middle qubit by $\frac{3}{2\pi}$ and the least significant qubit by $3\pi$.

In [20]:
n = 3
qt_state = tq.gates.H(target=0) + tq.gates.H(target=1) + tq.gates.H(target=2) + tq.gates.Phase(target=0, phi=n* pi/4) \
+ tq.gates.Phase(target=1, phi=n * pi/2) + tq.gates.Phase(target=2, phi=n * pi)
tq.draw(qt_state)

     ┌───┐┌──────────┐
q_0: ┤ H ├┤ RZ(3π/4) ├
     ├───┤├──────────┤
q_1: ┤ H ├┤ RZ(3π/2) ├
     ├───┤└┬────────┬┘
q_2: ┤ H ├─┤ RZ(3π) ├─
     └───┘ └────────┘ 
c: 3/═════════════════
                      


Simulating the $QFT^\dagger$ circuit should get us back to the original state of $\lvert 011 \rangle$

In [21]:
print(tq.simulate(qt_state + qft(3).dagger(), samples=100, read_out_qubits=[2,1,0]))

+100.0000|011> 


After a $100$ sample simulations, we got the state  $\lvert 011 \rangle$ every time, as desired.

# 3. Quantum Phase Estimation <a id='qpe'></a>

## 3. 1 Mathematical Description of the Quantum Phase Estimation <a id='math2'></a>

This circuit is aimed at estimating the phase of a unitary operator $U$. It estimates $\theta$ in $U\vert\psi \rangle =e^{\boldsymbol{2\pi i} \theta }|\psi \rangle$, where $|\psi\rangle$ is an eigenvector and $e^{\boldsymbol{2\pi i}\theta}$ is the corresponding eigenvalue. The circuit operates in the following steps:



 

<li> First we apply a $n$-bit Hadamard gate operation $H^{\otimes n}$ on the $n$-counting register: 



$$ \psi_1 = {\frac {1}{2^{\frac {n}{2}}}}\left(|0\rangle +|1\rangle \right)^{\otimes n} \lvert \psi \rangle$$



<li> $\textbf{Controlled Unitary Operations}$: Like the controlled phase rotation gate, the controlled unitary $C-U$ applies the unitary operator $U$ on the target register if and only if its corresponding control bit is $|1\rangle$. Since $U$ is a unitary operatory with eigenvector $|\psi\rangle$ such that $U|\psi \rangle =e^{\boldsymbol{2\pi i} \theta }|\psi \rangle$, we get the following:


$$U^{2^{j}}|\psi \rangle =U^{2^{j}-1}U|\psi \rangle =U^{2^{j}-1}e^{2\pi i\theta }|\psi \rangle =\cdots =e^{2\pi i2^{j}\theta }|\psi \rangle$$



Applying all the $n$ controlled operations $C − U^{2^j}$ with $0\leq j\leq n-1$, and using the relation $|0\rangle \otimes |\psi \rangle +|1\rangle \otimes e^{2\pi i\theta }|\psi \rangle =\left(|0\rangle +e^{2\pi i\theta }|1\rangle \right)\otimes |\psi \rangle$, carefully noting the relationship between the powers of $e^{\boldsymbol{2\pi i} \theta}$ and the binary represention of each qubit, we get the following:

\begin{aligned}
\psi_{2} =\frac {1}{2^{\frac {n}{2}}} \left(|0\rangle+{e^{\boldsymbol{2\pi i} \theta 2^{n-1}}}|1\rangle \right) \otimes \cdots \otimes \left(|0\rangle+{e^{\boldsymbol{2\pi i} \theta 2^{1}}}\vert1\rangle \right) \otimes \left(|0\rangle+{e^{\boldsymbol{2\pi i} \theta 2^{0}}}\vert1\rangle \right) \otimes |\psi\rangle  = \frac{1}{2^{\frac {n}{2}}}\sum _{k=0}^{2^{n}-1}e^{\boldsymbol{2\pi i} \theta k}|k\rangle \otimes \vert\psi\rangle
\end{aligned}
where $k$ denotes the integer representation of n-bit binary numbers. 

<li> $\textbf{Inverse Fourier Transform}$: Recall from above that QFT maps an n-qubit input state $\vert x\rangle$ into an output as

$$
QFT\vert x \rangle = \frac{1}{2^\frac{n}{2}}
\left(\vert0\rangle + e^{\frac{2\pi i}{2}x} \vert1\rangle\right) 
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^2}x} \vert1\rangle\right) 
\otimes  
\ldots
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^{n-1}}x} \vert1\rangle\right) 
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^n}x} \vert1\rangle\right) 
$$

Replacing $x$ by $2^n\theta$ in the above expression gives exactly the expression derived in step 2 above. Therefore, to recover the state $\vert2^n\theta\rangle$, apply an inverse Fourier transform on the n-bit counting registers. Doing so get us:

$$
QFT^\dagger QFT\vert 2^n \theta \rangle =  \vert 2^n \theta \rangle = QFT^\dagger(\frac{1}{2^\frac{n}{2}}
\left(\vert0\rangle + e^{\frac{2\pi i}{2}x} \vert1\rangle\right) 
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^2}x} \vert1\rangle\right) 
\otimes  
\ldots
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^{n-1}}x} \vert1\rangle\right) 
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^n}x} \vert1\rangle\right)) 
$$

<li> $\textbf{Measurement}$: Hence, cleary the above set of actions when measured should peak at $x = 2^n\theta$. For the case when $2^n\theta$ is an integer, measuring in the computational basis gives the phase in the n-bit counting registers with high probability. For the case when $2^n\theta$ is not an integer, it can be shown that the above expression still peaks near $x = 2^n\theta$ with probability better than $4/\pi^2 \approx 40\%$.

## 3.2 Quantum Phase Estimation Circuit <a id='sim2'></a>

Let us consider an example where we apply the $T$-gate:
$$ T|1\rangle = 
\begin{bmatrix}
1 & 0\\
0 & e^\frac{i\pi}{4}\\ 
\end{bmatrix}
\begin{bmatrix}
0\\
1\\ 
\end{bmatrix}
= e^\frac{i\pi}{4}|1\rangle $$

thus the expected results of the QPE should give us $\theta = \frac{1}{8}$.

Now let us implement and apply QPE:

In [23]:
from typing import Any

def qft_dagger(n):
    """Given the number of qubit registers <n>, return the QFT^dagger circuit."""
    return qft(n).dagger()

def qpe(psi : 'QCircuit', n : int, angle : float):
    """
    Given an eigenvector <psi> and the number of counting registers <n>,
    perform the Quantum Phase Algorithm and phase rotation angle <angle>
    """
    circuit = psi
    for i in range(n):
        circuit += tq.gates.H(target=i)
    repetitions = 1
    for counting_qubit in range(n):
        for i in range(repetitions):
            circuit += tq.gates.Phase(angle, counting_qubit, n)
        repetitions *= 2
    circuit += qft_dagger(3)
    return circuit

psi = tq.gates.X(target=3)
angle = pi/4
circuit = qpe(psi, 3, angle)

tq.draw(circuit)

     ┌───┐┌─────────┐┌───┐┌──────────┐┌───┐                                 »
q_0: ┤ H ├┤ RZ(π/8) ├┤ X ├┤ RZ(-π/8) ├┤ X ├─────────────────────────────────»
     ├───┤├─────────┤└─┬─┘└──────────┘└─┬─┘           ┌───┐┌──────────┐┌───┐»
q_1: ┤ H ├┤ RZ(π/8) ├──┼────────────────┼─────────────┤ X ├┤ RZ(-π/8) ├┤ X ├»
     ├───┤├─────────┤  │                │             └─┬─┘└──────────┘└─┬─┘»
q_2: ┤ H ├┤ RZ(π/8) ├──┼────────────────┼───────────────┼────────────────┼──»
     ├───┤├─────────┤  │                │  ┌─────────┐  │                │  »
q_3: ┤ X ├┤ RZ(π/8) ├──■────────────────■──┤ RZ(π/8) ├──■────────────────■──»
     └───┘└─────────┘                      └─────────┘                      »
c: 4/═══════════════════════════════════════════════════════════════════════»
                                                                            »
«                                                                        »
«q_0: ─────────────────────────────────────────────────────────────

In [24]:
print(tq.simulate(circuit, samples=100, read_out_qubits=[2,1,0]))

+100.0000|001> 


So the results of the QPE algorithm gave us, with hundred percent accuracy, the state $\lvert 001 \rangle$. Thus, we get that $2^n\theta = 2^3\theta = b(001) = 1$, which gives us:
$$ \theta = \frac{1}{8}$$
as expected.

# 4. References <a id='references'></a>
1. Quantum Fourier Transform. (2020). Retrieved 21 December 2020, from https://qiskit.org/textbook/ch-algorithms/quantum-fourier-transform.html
2. Quantum Fourier Tranform. (2020). Retrieved 21 December 2020, from https://viterbi-web.usc.edu/~tbrun/Course/lecture13.pdf
3. Quantum Phase Estimation. (2020). Retrieved 31 January 2021, from https://qiskit.org/textbook/ch-algorithms/quantum-fourier-transform.html